In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm 
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

MODEL_NAME = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
MODEL_PATH = f"../models/{MODEL_NAME}"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.save_pretrained(MODEL_PATH)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    local_files_only=True
)

/opt/homebrew/Caskroom/miniforge/base/envs/sentiment_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 618/618 [00:00<00:00, 1.18MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:01<00:00, 2.87MB/s]
tokenizer.json: 100%|██████████| 9.10M/9.10M [00:02<00:00, 4.14MB/s]
config.json: 100%|██████████| 992/992 [00:00<00:00, 2.04MB/s]
pytorch_model.bin:   3%|▎         | 31.5M/1.11G [00:04<02:12, 8.15MB/s]Error while downloading from https://cdn-lfs.huggingface.co/arianpasquali/twitter-xlm-roberta-base-sentiment-finetunned/4b9f7e2e7dd748c52a1dd3805f3fd1bb74c80fbfbb1167a6454bf02f46205b6f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-strea

In [2]:
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        # t = '' if t.startswith(('@', 'http')) and len(t) > 1 else t
        new_text.append(t)
    return " ".join(new_text)

def infer(text):
    labels = ['negative', 'neutral', 'positive']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    output = ranking[0]
    label = labels[output]
    return text, label

In [3]:
def evaluate(predictions, true_labels):
    cls_report = classification_report(
        true_labels, 
        predictions
    )
    cnf_matrix = confusion_matrix(
        true_labels, 
        predictions
    )
    accuracy = accuracy_score(true_labels, predictions)

    print("Classification Report:\n", cls_report)
    print("Confusion Matrix:\n", cnf_matrix)
    print("Accuracy: ", accuracy)


def test_batch(csv_path):
    df = pd.read_csv(csv_path)
    data = df['text'].tolist()
    true_labels = df['expected_sentiment'].tolist()
    predictions = []

    for i in tqdm(range(len(data)), desc="Processing"):
        text = data[i]
        true_label = true_labels[i]
        processed_text, predicted_label = infer(text)
        predictions.append(predicted_label)

    evaluate(predictions, true_labels)

In [4]:
csv_path = '../data/sentiment_test_cases.csv'
test_batch(csv_path)

Processing:   0%|          | 0/498 [00:00<?, ?it/s]

Processing: 100%|██████████| 498/498 [00:19<00:00, 25.62it/s]

Classification Report:
               precision    recall  f1-score   support

    negative       0.89      0.64      0.75       177
     neutral       0.55      0.94      0.70       139
    positive       0.89      0.65      0.76       182

    accuracy                           0.73       498
   macro avg       0.78      0.75      0.73       498
weighted avg       0.80      0.73      0.74       498

Confusion Matrix:
 [[114  51  12]
 [  6 131   2]
 [  8  55 119]]
Accuracy:  0.7309236947791165
